# Training set

In [132]:
import yfinance as yf
import pandas as pd
import numpy as np

In [133]:
start, end = "2006-01-01", "2018-01-01"
Russell = yf.download("^RUA", start=start, end=end, progress= False)

In [134]:
vix = pd.read_excel("VIXCLS.xls")
m2 = pd.read_excel("WM2NS.xls")
unemp_rate = pd.read_excel("U2RATE.xls")
con_sentiment = pd.read_excel("UMCSENT.xls")
t10y2y = pd.read_excel("T10Y2Y.xls")
ppi = pd.read_excel("PPIACO.xls")
fed = pd.read_excel('FEDFUNDS.xls')
cpi = pd.read_excel("CPIAUCSL.xls")
dff = pd.read_excel('DFF.xls')

vix = vix.set_index('observation_date')
m2 = m2.set_index('observation_date')
unemp_rate = unemp_rate.set_index('observation_date')
con_sentiment = con_sentiment.set_index('observation_date')
t10y2y = t10y2y.set_index('observation_date')
ppi = ppi.set_index('observation_date')
fed = fed.set_index('observation_date')
cpi = cpi.set_index('observation_date')
dff = dff.set_index('observation_date')


vix = vix.resample('D').ffill()
m2 = m2.resample('D').ffill()
unemp_rate = unemp_rate.resample('D').ffill()
con_sentiment = con_sentiment.resample('D').ffill()
t10y2y = t10y2y.resample('D').ffill()
ppi = ppi.resample('D').ffill()
fed = fed.resample('D').ffill()
dff = dff.resample('D').ffill()
cpi = cpi.resample('D').ffill()


In [135]:
Russell = pd.merge(Russell,vix,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,m2,how='left',left_index=True, right_index=True)

Russell = pd.merge(Russell,unemp_rate,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,con_sentiment,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,t10y2y,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,ppi,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,fed,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,dff,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,cpi,how='left',left_index=True, right_index=True)

In [136]:
Russell = Russell.drop(['Adj Close','Volume'],axis=1)

In [137]:
Russell.to_csv('data.csv')

In [138]:
Russell

,Open,High,Low,Close,VIXCLS,WM2NS,unemployement_rate,consumer_sentiment,T10Y2Y,PPIACO,FEDFUNDS,DFF,CPIAUCSL
Date,,,,,,,,,,,,,
2006-01-03,723.309998,735.590027,720.849976,734.719971,11.14,6731.0,2.2,91.2,0.03,164.3,4.29,4.34,199.300
2006-01-04,734.719971,738.750000,734.320007,737.909973,11.37,6731.0,2.2,91.2,0.05,164.3,4.29,4.22,199.300
2006-01-05,737.909973,739.869995,736.200012,738.369995,11.31,6731.0,2.2,91.2,0.04,164.3,4.29,4.24,199.300
2006-01-06,738.369995,745.820007,738.369995,745.400024,11.00,6731.0,2.2,91.2,0.02,164.3,4.29,4.22,199.300
2006-01-09,745.400024,749.330017,745.260010,748.750000,11.13,6749.7,2.2,91.2,0.02,164.3,4.29,4.25,199.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,1589.069946,1589.699951,1585.420044,1588.250000,9.90,13965.8,2.0,95.9,0.57,196.3,1.30,1.42,247.805
2017-12-26,1587.520020,1588.300049,1585.630005,1587.000000,10.25,13934.4,2.0,95.9,0.55,196.3,1.30,1.42,247.805
2017-12-27,1587.189941,1590.280029,1586.300049,1588.160034,10.47,13934.4,2.0,95.9,0.53,196.3,1.30,1.42,247.805


In [139]:
window_size = 20
# 计算移动平均
Russell['movingaverage'] = Russell['Close'].rolling(window=window_size).mean()

# 计算移动平均的百分比变化
Russell['pct_close'] = Russell['movingaverage'].pct_change()

# 根据条件设置 'label' 列
Russell['label'] = Russell['pct_close'].apply(lambda x: 1 if x > 0.0001 else (-1 if x < -0.0001 else 0)).shift(periods=-1)
Russell = Russell.dropna()
Russell.head(50)

C:\Users\Zaichuan\AppData\Local\Temp\ipykernel_66000\2154565707.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  Russell['pct_close'] = Russell['movingaverage'].pct_change()


,Open,High,Low,Close,VIXCLS,WM2NS,unemployement_rate,consumer_sentiment,T10Y2Y,PPIACO,FEDFUNDS,DFF,CPIAUCSL,movingaverage,pct_close,label
Date,,,,,,,,,,,,,,,,
2006-02-01,746.770020,748.530029,744.820007,748.039978,12.36,6639.2,2.2,86.7,-0.02,161.8,4.49,4.47,199.4,743.795999,0.000896,1.0
2006-02-02,748.039978,748.039978,739.270020,741.229980,13.23,6639.2,2.2,86.7,-0.02,161.8,4.49,4.48,199.4,743.962000,0.000223,0.0
2006-02-03,741.229980,741.390015,735.479980,737.390015,12.96,6639.2,2.2,86.7,-0.05,161.8,4.49,4.51,199.4,743.913000,-0.000066,-1.0
2006-02-06,737.400024,739.039978,735.929993,738.340027,13.04,6705.7,2.2,86.7,-0.07,161.8,4.49,4.51,199.4,743.560001,-0.000475,-1.0
2006-02-07,738.340027,738.940002,731.270020,731.830017,13.59,6705.7,2.2,86.7,-0.04,161.8,4.49,4.47,199.4,742.714001,-0.001138,-1.0
2006-02-08,731.830017,737.929993,731.330017,737.409973,12.83,6705.7,2.2,86.7,-0.08,161.8,4.49,4.48,199.4,742.121500,-0.000798,-1.0
2006-02-09,737.409973,742.770020,735.619995,736.159973,13.12,6705.7,2.2,86.7,-0.12,161.8,4.49,4.52,199.4,741.358499,-0.001028,-1.0
2006-02-10,736.159973,739.000000,730.530029,737.469971,12.87,6705.7,2.2,86.7,-0.10,161.8,4.49,4.51,199.4,740.897498,-0.000622,-1.0
2006-02-13,737.469971,737.469971,731.859985,734.229980,13.35,6709.0,2.2,86.7,-0.10,161.8,4.49,4.44,199.4,740.221497,-0.000912,-1.0


In [140]:
Record = {}
for a in Russell.drop(columns=['label']):
    Record[a] = {}
    Record[a]['Max'] = max(Russell[a])
    Record[a]['Min'] = min(Russell[a])
    Russell[a] = (Russell[a]-min(Russell[a]))/(max(Russell[a])-min(Russell[a]))
Record

C:\Users\Zaichuan\AppData\Local\Temp\ipykernel_66000\3693363192.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Russell[a] = (Russell[a]-min(Russell[a]))/(max(Russell[a])-min(Russell[a]))


{'Open': {'Max': 1592.3399658203125, 'Min': 389.5899963378906},
 'High': {'Max': 1594.3299560546875, 'Min': 400.5},
 'Low': {'Max': 1587.949951171875, 'Min': 384.4100036621094},
 'Close': {'Max': 1591.9599609375, 'Min': 389.6099853515625},
 'VIXCLS': {'Max': 80.86, 'Min': 9.14},
 'WM2NS': {'Max': 13965.8, 'Min': 6639.2},
 'unemployement_rate': {'Max': 6.5, 'Min': 2.0},
 'consumer_sentiment': {'Max': 100.7, 'Min': 55.3},
 'T10Y2Y': {'Max': 2.91, 'Min': -0.19},
 'PPIACO': {'Max': 208.3, 'Min': 161.8},
 'FEDFUNDS': {'Max': 5.26, 'Min': 0.07},
 'DFF': {'Max': 5.41, 'Min': 0.04},
 'CPIAUCSL': {'Max': 247.805, 'Min': 199.4},
 'movingaverage': {'Max': 1576.092999267578, 'Min': 425.0834991455078},
 'pct_close': {'Max': 0.010589882052007393, 'Min': -0.01743746555807546}}

In [141]:
X_train = Russell.drop(columns=['Open', 'High', 'Low', 'Close', 'movingaverage', 'pct_close', 'label']).values
y_train = Russell['label'].values
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{-1.0: 997, 0.0: 120, 1.0: 1860}

# Testing set

In [142]:
start, end = "2018-01-01", "2022-01-01"
Russell = yf.download("^RUA", start=start, end=end, progress= False)

In [143]:
vix = pd.read_excel("VIXCLS.xls")
m2 = pd.read_excel("WM2NS.xls")
unemp_rate = pd.read_excel("U2RATE.xls")
con_sentiment = pd.read_excel("UMCSENT.xls")
t10y2y = pd.read_excel("T10Y2Y.xls")
ppi = pd.read_excel("PPIACO.xls")
fed = pd.read_excel('FEDFUNDS.xls')
cpi = pd.read_excel("CPIAUCSL.xls")
dff = pd.read_excel('DFF.xls')

vix = vix.set_index('observation_date')
m2 = m2.set_index('observation_date')
unemp_rate = unemp_rate.set_index('observation_date')
con_sentiment = con_sentiment.set_index('observation_date')
t10y2y = t10y2y.set_index('observation_date')
ppi = ppi.set_index('observation_date')
fed = fed.set_index('observation_date')
cpi = cpi.set_index('observation_date')
dff = dff.set_index('observation_date')


vix = vix.resample('D').ffill()
m2 = m2.resample('D').ffill()
unemp_rate = unemp_rate.resample('D').ffill()
con_sentiment = con_sentiment.resample('D').ffill()
t10y2y = t10y2y.resample('D').ffill()
ppi = ppi.resample('D').ffill()
fed = fed.resample('D').ffill()
dff = dff.resample('D').ffill()
cpi = cpi.resample('D').ffill()

In [144]:
Russell = pd.merge(Russell,vix,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,m2,how='left',left_index=True, right_index=True)

Russell = pd.merge(Russell,unemp_rate,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,con_sentiment,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,t10y2y,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,ppi,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,fed,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,dff,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,cpi,how='left',left_index=True, right_index=True)

In [145]:
Russell = Russell.drop(['Adj Close','Volume'],axis=1)
window_size = 20
# 计算移动平均
Russell['movingaverage'] = Russell['Close'].rolling(window=window_size).mean()

# 计算移动平均的百分比变化
Russell['pct_close'] = Russell['movingaverage'].pct_change()

# 根据条件设置 'label' 列
Russell['label'] = Russell['pct_close'].apply(lambda x: 1 if x > 0.0001 else (-1 if x < -0.0001 else 0)).shift(periods=-1)
Russell = Russell.dropna()
Russell.head(50)

C:\Users\Zaichuan\AppData\Local\Temp\ipykernel_66000\77827842.py:7: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  Russell['pct_close'] = Russell['movingaverage'].pct_change()


,Open,High,Low,Close,VIXCLS,WM2NS,unemployement_rate,consumer_sentiment,T10Y2Y,PPIACO,FEDFUNDS,DFF,CPIAUCSL,movingaverage,pct_close,label
Date,,,,,,,,,,,,,,,,
2018-01-31,1664.949951,1674.709961,1658.819946,1664.680054,13.54,13745.1,2.0,95.7,0.58,197.9,1.41,1.34,248.859,1650.664001,0.002089,1.0
2018-02-01,1664.199951,1671.630005,1659.119995,1664.369995,13.47,13745.1,2.0,99.7,0.62,199.3,1.42,1.42,249.529,1653.623004,0.001793,1.0
2018-02-02,1660.520020,1660.520020,1628.540039,1629.560059,17.31,13745.1,2.0,99.7,0.69,199.3,1.42,1.42,249.529,1654.550006,0.000561,-1.0
2018-02-05,1627.119995,1630.130005,1560.310059,1564.660034,37.32,13839.2,2.0,99.7,0.69,199.3,1.42,1.42,249.529,1651.714508,-0.001714,-1.0
2018-02-06,1562.699951,1593.530029,1538.540039,1589.780029,29.98,13839.2,2.0,99.7,0.69,199.3,1.42,1.42,249.529,1649.986011,-0.001046,-1.0
2018-02-07,1589.780029,1607.920044,1583.119995,1583.180054,27.73,13839.2,2.0,99.7,0.69,199.3,1.42,1.42,249.529,1647.849011,-0.001295,-1.0
2018-02-08,1584.150024,1584.630005,1525.260010,1525.260010,33.46,13839.2,2.0,99.7,0.72,199.3,1.42,1.42,249.529,1642.914014,-0.002995,-1.0
2018-02-09,1525.780029,1557.130005,1496.939941,1546.550049,29.06,13839.2,2.0,99.7,0.78,199.3,1.42,1.42,249.529,1638.372516,-0.002764,-1.0
2018-02-12,1547.520020,1576.510010,1547.000000,1567.160034,25.61,13835.0,2.0,99.7,0.77,199.3,1.42,1.42,249.529,1634.362018,-0.002448,-1.0


In [146]:
for a in Russell.drop(columns=['label']):
    Russell[a] = (Russell[a]-Record[a]['Min'])/(Record[a]['Max']-Record[a]['Min'])
X_test = Russell.drop(columns=['Open', 'High', 'Low', 'Close', 'movingaverage', 'pct_close', 'label']).values
y_test = Russell['label'].values
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

{-1.0: 269, 0.0: 29, 1.0: 652}

# Training classifier

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'n_estimators': [50, 125, 250, 1000],
    'max_depth': [3, 5, 7, 10, 15, 20],
    'min_samples_leaf': [1, 3, 5, 10]
}

classifier = RandomForestClassifier()
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=3, n_jobs=-1, verbose=3)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

Fitting 3 folds for each of 96 candidates, totalling 288 fits


In [148]:
classifier = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], min_samples_leaf=best_params['min_samples_leaf'])
classifier.fit(X_train, y_train)
y_hat = classifier.predict(X_test)
accuracy_score(y_test, y_hat)

0.7273684210526316